In [1]:
# importing the necessary libraries
import urllib.request
import re
from importlib.metadata import version
import tiktoken

In [2]:
# # ---- Reading the text file from the URL of the text file ----
# url = ("https://raw.githubusercontent.com/rasbt/"
#       "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
#       "the-verdict.txt")
# file_path = "the-verdict.txt"
# urllib.request.urlretrieve(url, file_path)

# with open("the-verdict.txt", "r", encoding="utf-8") as f:
#    raw_text = f.read()
# print("Total number of character:", len(raw_text))
# # print(raw_text[:99])

In [3]:
# # ---- Preprocessing the text to split the text into tokens ----
# preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
# preprocessed = [item.strip() for item in preprocessed if item.strip()]
# print(len(preprocessed))
# # print(preprocessed[:30])

In [4]:
# # ---- Building a vocabulary from the preprocessed (tokenised) text ----
# all_words = sorted(set(preprocessed))
# all_words.extend(["<|endoftext|>", "<|unk|>"])
# vocab = {token:integer for integer,token in enumerate(all_words)}

# class SimpleTokenizerV2:
#       def __init__(self, vocab):
#             self.str_to_int = vocab
#             self.int_to_str = {i:s for s,i in vocab.items()}

#       def encode(self, text):
#             preprocessed = re.split(r'([,.?_!"()\']|--|\s)', text)
#             preprocessed = [item.strip() for item in preprocessed if item.strip()]
#             preprocessed = [item if item in self.str_to_int else "<|unk|>" for item in preprocessed]

#             ids = [self.str_to_int[s] for s in preprocessed]
#             return ids

#       def decode(self, ids):
#             text = " ".join([self.int_to_str[i] for i in ids])
#             text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
#             return text

# # tokenizer = SimpleTokenizerV1(vocab)
# # text =  """"It's the last he painted, you know," Mrs. Gisburn said with pardonable pride."""
# # ids = tokenizer.encode(text)
# # print(tokenizer.decode(ids))

# text1 = "Hello, do you like tea?"
# text2 = "In the sunlit terraces of the palace."
# text3 = "painted."
# text4 = "pardonable."
# text5 = "lmao."
# text = " <|endoftext|> ".join((text1, text2, text3, text4, text5))
# # print(text)
# tokenizer = SimpleTokenizerV2(vocab)
# # print(tokenizer.encode(text))
# # print(tokenizer.decode(tokenizer.encode(text)))

In [5]:
# --- Using tiktoken to implement BPE ----

# print("tiktoken version:", version("tiktoken"))
# tokenizer = tiktoken.encoding_for_model("gpt-3.5-turbo")
# text = ("Hello, do you like tea? <|endoftext|> In the sunlit terraces"
#         "of Akwirw ier." )
# integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"}, disallowed_special=())
# print(integers)
# strings = tokenizer.decode(integers)
# print(strings)

# # ---- create input-target pairs ----
# with open("the-verdict.txt", "r", encoding="utf-8") as f:
#       raw_text = f.read()

# enc_text = tokenizer.encode(raw_text)
# print(len(enc_text))

# enc_sample = enc_text[50:]

# context_size = 4
# x = enc_sample[:context_size]
# y = enc_sample[1:context_size+1]
# print(f"x: {x}")
# print(f"y:      {y}")

# for i in range(1, context_size+1):
#       context = enc_sample[:i]
#       desired = enc_sample[i]
#       print(context, "---->", desired)

# for i in range(1, context_size+1):
#       context = enc_sample[:i]
#       desired = enc_sample[i]
#       print(tokenizer.decode(context), "---->", tokenizer.decode([desired]))

In [6]:
# using PyTorch dataset

import torch
from torch.utils.data import Dataset, DataLoader
# torch.__version__

class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []
        token_ids = tokenizer.encode(txt)   

        for i in range(0, len(token_ids) - max_length, stride):    
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):   
        return len(self.input_ids)
    
    def __getitem__(self, idx):        
        return self.input_ids[idx], self.target_ids[idx]


In [7]:
# creating PyToch DataLoader using the GPTDatasetV1 class along with BPE tiktoken tokenizer 

def create_dataloader_v1(txt, batch_size=4, max_length=256, 
                         stride = 128, shuffle = True, drop_last = True, num_workers=0):
    tokenizer = tiktoken.get_encoding("gpt2")                        
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)  
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,    
        num_workers=num_workers    
    )
    return dataloader

In [8]:
# basic example to use the dataloader

# sometimes, takes a long time to execute (more than 1 minute, idk why)

with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

dataloader = create_dataloader_v1(
    raw_text, batch_size=4, max_length=4, stride=4, shuffle=False)

data_iter = iter(dataloader)     
first_batch = next(data_iter)
print(first_batch)


[tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257]]), tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922]])]


In [9]:
vocab_size = 50257
output_dim = 256
token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

max_length = 4
dataloader = create_dataloader_v1(
   raw_text, batch_size=8, max_length=max_length,
  stride=max_length, shuffle=False
)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Token IDs:\n", inputs)
print("\nInputs shape:\n", inputs.shape)

token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)

Token IDs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Inputs shape:
 torch.Size([8, 4])
torch.Size([8, 4, 256])


In [10]:
# For a GPT model’s absolute embedding approach, we just need to create another
#  embedding layer that has the same embedding dimension as the token_embedding_
#  layer:  The input to the pos_embeddings is usually a placeholder vector torch.arange(con
# text_length), which contains a sequence of numbers 0, 1, ..., up to the maximum
#  input length –1. The context_length is a variable that represents the supported input
#  size of the LLM. Here,
# For a GPT model’s  we choose it similar to the maximum length of the input text.
#  In practice, input text can be longer than the supported context length, in which case
#  we have to truncate the text.

context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)
pos_embeddings = pos_embedding_layer(torch.arange(context_length))
print(pos_embeddings.shape)

input_embeddings = token_embeddings + pos_embeddings
print(input_embeddings.shape)

# we now have the input embeddings that are used as input for the main LLM layers

torch.Size([4, 256])
torch.Size([8, 4, 256])


In [11]:
# 3-dimensional embeddings of the sequence “Your journey starts with one step”

import torch
inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your (x^1)
   [0.55, 0.87, 0.66], # journey (x^2)
   [0.57, 0.85, 0.64], # starts (x^3)
   [0.22, 0.58, 0.33], # with (x^4)   
   [0.77, 0.25, 0.10], # one (x^5)   
   [0.05, 0.80, 0.55]] # step (x^6)   
)

# attention scores for 2nd input ("journey") wrt to all other inputs - using dot products
# mutliplies element wise i.e. 1st ele with 1st ele, 2nd with 2nd and so on
query = inputs[1]                           
attn_scores_2 = torch.empty(inputs.shape[0])
for i, x_i in enumerate(inputs):
    attn_scores_2[i] = torch.dot(x_i, query)
print(attn_scores_2)

# normalize the attention scores to convert them to attention weights
attn_weights_2 = torch.softmax(attn_scores_2, dim=0)
print("Attention weights:", attn_weights_2)
print("Sum:", attn_weights_2.sum())

# computing z(2) by multiplying the embedded input tokens, x(i), with the corresponding attention weights and then summing the resulting vectors
query = inputs[1]        
context_vec_2 = torch.zeros(query.shape)
for i,x_i in enumerate(inputs):
    context_vec_2 += attn_weights_2[i]*x_i
print(context_vec_2)



tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])
Attention weights: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum: tensor(1.)
tensor([0.4419, 0.6515, 0.5683])


In [13]:
attn_scores = inputs @ inputs.T
attn_weights = torch.softmax(attn_scores, dim=-1)

all_context_vecs = attn_weights @ inputs
print(all_context_vecs)

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])


In [17]:
# now with trainable weights
# we compute 3 matrices: query Wq (for current input), key Wk, value Wv
# taking 2nd input as query
x_2 = inputs[1]    
d_in = inputs.shape[1]     
d_out = 2
# Note that in GPT-like models, the input and output dimensions are usually the same, but to better follow the computation, we’ll use different input (d_in=3) and output (d_out=2) dimensions here.

W_query = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False) 
W_key   = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False) 
W_value = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)

query_2 = x_2 @ W_query
keys = inputs @ W_key 
values = inputs @ W_value

attn_scores_2 = query_2 @ keys.T      
print(attn_scores_2)

tensor([0.8842, 1.2164, 1.2144, 0.6294, 0.8355, 0.6955])
